In [1]:
# https://spell.ml/blog/gradient-checkpointing-pytorch-YGypLBAAACEAefHs

In [2]:
import torch
import torch.nn as nn

from torch.utils.checkpoint import checkpoint_sequential

In [3]:
# a trivial model
model = nn.Sequential(
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 20),
    nn.ReLU(),
    nn.Linear(20, 5),
    nn.ReLU()
)

In [8]:
# # model input
# input_var = torch.randn(1, 100, requires_grad=True)

# # the number of segments to divide the model into
# segments = 2

# # finally, apply checkpointing to the model
# # note the code that this replaces:
# # out = model(input_var)
# out = checkpoint_sequential(modules, segments, input_var)

# # backpropagate
# out.sum().backwards()

In [29]:
class CIFAR10Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_block_1 = nn.Sequential(*[
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        ])
        self.dropout_1 = nn.Dropout(0.25)
        self.cnn_block_2 = nn.Sequential(*[
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        ])
        self.dropout_2 = nn.Dropout(0.25)
        self.flatten = lambda inp: torch.flatten(inp, 1)
        self.linearize = nn.Sequential(*[
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU()
        ])
        self.dropout_3 = nn.Dropout(0.5)
        self.out = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, X):
        X = self.cnn_block_1(X)
        X = self.dropout_1(X)
        X = torch.utils.checkpoint.checkpoint(self.cnn_block_2, X)
        X = self.dropout_2(X)
        X = self.flatten(X)
        X = self.linearize(X)
        X = self.dropout_3(X)
        X = self.out(X)
        X = self.softmax(X)
        return X

In [42]:
images = torch.randn(8,3,32,32,  requires_grad=True)
label = torch.randint(10, (8,))

In [43]:
model = CIFAR10Model()
Celoss = nn.CrossEntropyLoss()


In [44]:
out = model(images)

In [45]:
label

tensor([8, 7, 2, 0, 0, 3, 4, 9])

In [46]:
loss = Celoss(out, label)

In [49]:
loss.backward()